
<center>
    <h2 style="color: #f0f2f3; background-color: #343d40; font-size: 50px; font-family: monospace; letter-spacing: 2px; padding:20px;">Student Development Analysis</h2>
</center> 


In [20]:
import pandas as pd
from datetime import date, datetime
import plotly.io as pio
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [22]:

df = pd.read_excel("C:\\Users\\USER\\OneDrive\\Desktop\\DataSet.xlsx", sheet_name = 'StudentData')
df_archive = pd.read_excel("C:\\Users\\USER\\OneDrive\\Desktop\\DataSet.xlsx", sheet_name = 'ArchivedData')
#print(df)

print(df.shape)
print(df_archive.shape)

(38, 21)
(138, 15)




<h2 style="color: #f0f2f3; background-color: #343d40; font-size: 50px; font-family: monospace; letter-spacing: 2px; padding:20px;">Part 1: Student Analysis</h2>



In [26]:

def strengths(student):
    marks = student.iloc[:,9:19].to_dict('records')[0]
    marks = sorted(marks.items(), key = lambda x: x[1], reverse = True)
    strength = marks[:3]
    #print(strength)
    return strength[0], strength[1], strength[2]

def weaknesses(student):
    marks = student.iloc[:,9:19].to_dict('records')[0]
    marks = sorted(marks.items(), key = lambda x: x[1])
    weakness = marks[:3]
    #print(weakness)
    return weakness[0], weakness[1], weakness[2]

def calculateRank(student):
    cls = student['Class'].to_numpy()[0]
    name = student['Name'].to_numpy()[0]
    students = df[df['Class'] == cls]
    students['Rank'] = students['Total'].rank(method='min', ascending=False).astype(int)
    return students[students['Name'] == name]['Rank'].to_numpy()[0]

def assign_grade(avg):
    if avg >= 95:
        return 'S'
    elif avg >= 90:
        return 'A'
    elif avg >= 80:
        return 'B'
    elif avg >= 70:
        return 'C'
    elif avg >= 60:
        return 'D'
    else:
        return 'F'

def getGradeCard(student):
    frame = student.iloc[:,9:]
    frame['Grade'] = frame['Average'].apply(assign_grade)
    frame_dict = frame.to_dict('split')
    return frame_dict


In [28]:

app = dash.Dash(
    __name__,external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True
)


def createStudentCard(std_id):
    if std_id == None:
        return " "
    student = df[df['Student ID'] == std_id]
    
    name = student['Name'].to_numpy()[0]
    s_id = student['Student ID'].to_numpy()[0]
    gender = student['Gender'].to_numpy()[0]
    email = student['Email'].to_numpy()[0]
    contact = student['Phone Number'].to_numpy()[0]
    dob = pd.to_datetime(student['Date Of Birth'].to_numpy()[0], format = "%Y-%m-%d").date()
    age = student['Age'].to_numpy()[0]
    s_class = student['Class'].to_numpy()[0]
    profile_image = student['Profile Image'].to_numpy()[0]
    
    
    strength1, strength2, strength3 = strengths(student)
    weakness1, weakness2, weakness3 = weaknesses(student)
    rank = calculateRank(student)

    grade_card = getGradeCard(student)

    
    card = dbc.Container([
        dbc.Col([
            dbc.Container([
                dbc.Row([
                    dbc.Col([
                        dbc.Card([
                            #Name and ID Header
                            dbc.CardHeader([
                                dbc.Row([
                                    dbc.Col([
                                        html.H6(
                                            f"{name} ({s_id})",
                                            className="display-5 m-b-20 text-center",
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=12)
                                    ])
                            ], className="p-2", style={'background-color':' #132257','color':'#f0f2f3'}),
                            #Student Details Header
                            dbc.CardHeader([
                                dbc.Row([
                                    dbc.Col([
                                        html.H6(
                                            f"Gender: {gender}",
                                            className="m-b-20", 
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=5),
                                    dbc.Col([
                                        html.H6(
                                            f"DOB: {dob}({age})",
                                            className="m-b-20",
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=5),
                                    dbc.Col([
                                        html.H6(
                                            f"Class: {s_class}",
                                            className="m-b-20",
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=2),
                                    dbc.Col([
                                        html.H6(
                                            f"Email: {email}",
                                            className="m-b-20",
                                            style={'font-family': 'monospace','letter-spacing':'2px',}),
                                    ], width=5),
                                    dbc.Col([
                                        html.H6(
                                            f"Contact: {contact}",
                                            className="m-b-20",
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=5),
                                    dbc.Col([
                                        html.H6(
                                            f"Rank: {rank}",
                                            className="m-b-20",
                                            style={'font-family': 'monospace','letter-spacing':'2px'}),
                                    ], width=2)
                                ])
                            ],className="p-3 text-muted text-center", style={'background-color':'white','color':'black', 'font-size': '12px'}),
                            
                            dbc.Row([
                                #Profile Photo
                                dbc.Col([
                                    dbc.CardImg(src=profile_image,className="m-b-25 img-radius"),
                                ],width=5, style={'background-color': '#132257', 'border-radius':'20px'}),

                                #Card Body
                                dbc.Col([
                                    dbc.CardBody([
                                        #Stats
                                        html.H2(
                                            "Stats",
                                            className="m-b-20 text-bold",
                                            style={'font-family':'monospace', 'letter-spacing': '3px'}),
                                        html.Hr(),
                                        dbc.Row([
                                            #Strengths
                                            dbc.Col([
                                                html.H5(
                                                    "Strengths",
                                                    className="text-muted",
                                                    style={'font-family':'monospace', 'letter-spacing': '2px'}),
                                                html.Hr(),
                                                #Subjects and Progress Bars
                                                html.Span([
                                                    html.P(
                                                        strength1[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(strength1[1])}",
                                                        value=round(strength1[1]),
                                                        min=0,
                                                        max=100,
                                                        color="#132257",
                                                        style={"height": "15px"}),
                                                    
                                                    html.P(
                                                        strength2[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(strength2[1])}",
                                                        value=round(strength2[1]),
                                                        min=0,
                                                        max=100,
                                                        color="#132257",
                                                        style={"height": "15px"}),
                                                    html.P(
                                                        strength3[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(strength3[1])}",
                                                        value=round(strength3[1]),
                                                        min=0,
                                                        max=100,
                                                        color="#132257",
                                                        style={"height": "15px"}),
                                                ]),
                                            ],width=6,),
                                            
                                            #Weaknesses
                                            dbc.Col([
                                                html.H5(
                                                    "Weaknesses",
                                                    className="text-muted",
                                                    style={'font-family':'monospace', 'letter-spacing': '2px'}),
                                                html.Hr(),
                                                #Subjects and Progress Bars 
                                                html.Span([
                                                    html.P(
                                                        weakness1[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(weakness1[1])}",
                                                        value=round(weakness1[1]),
                                                        min=0,
                                                        max=100,
                                                        color="#6CABDD",
                                                        style={"height": "15px"}),
                                                    
                                                    html.P(
                                                        weakness2[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(weakness2[1])}",
                                                        value=round(weakness2[1]), 
                                                        min=0,
                                                        max=100, 
                                                        color="#6CABDD",
                                                        style={"height": "15px"}),
                                                    html.P(
                                                        weakness3[0],
                                                        className="f-w-600 text-bold",
                                                        style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                                                    dbc.Progress(
                                                        label=f"{round(weakness3[1])}",
                                                        value=round(weakness3[1]),
                                                        min=0,
                                                        max=100,
                                                        color="#6CABDD",
                                                        style={"height": "15px"}),
                                                ]),
                                            ],width=6,),
                                        ]),
                                        
                                        dbc.Row([
                                            #Total
                                            dbc.Col([
                                                html.H5(
                                                    "Total",
                                                    className="text-muted",
                                                    style={'font-family':'monospace', 'letter-spacing': '2px'}),
                                                html.Hr(),
                                                dbc.Card([
                                                    dbc.CardBody(
                                                        f"{float(student['Total'])}",
                                                        className="text-white display-6 f-w-200",
                                                        style={'font-family':'monospace', 'letter-spacing': '3px'} ),
                                                ], className="d-flex justify-content-center align-items-center", style={'border-radius':'7px', 'background-color': '#132257'}),
                                            ],width=6,),
                                            
                                            #Average
                                            dbc.Col([
                                                html.H5(
                                                    "Average",
                                                    className="text-muted",
                                                    style={'font-family':'monospace', 'letter-spacing': '2px'}),
                                                html.Hr(),
                                                dbc.Card([
                                                    dbc.CardBody(
                                                        f"{float(student['Average'])}",
                                                        className="text-white display-6 f-w-200",
                                                        style={'font-family':'monospace', 'letter-spacing': '3px'} ),
                                                ], className="d-flex justify-content-center align-items-center", style={'border-radius':'7px', 'background-color': '#6CABDD'}),
                                            ],width=6,),
                                        ],className="mt-5"),
                                    ],),
                                ],width=6,),
                                
                                #Grade Card
                                dbc.CardBody([
                                    #Heading
                                    dbc.Col([
                                        html.H2(
                                            "Grade Card",className="m-b-20 text-bold",
                                            style={'font-family':'monospace', 'letter-spacing': '3px'}),
                                        html.Hr(),
                                    ], className="p-3"),

                                    #Each Subject as list group items(subject name, marks, grade)
                                    dbc.Col([
                                       dbc.CardBody([
                                           dbc.Row([
                                               dbc.Col([
                                                   dbc.ListGroup([
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][0])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][0])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][0]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][1])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][1])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][1]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][2])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][2])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][2]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][3])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][3])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][3]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][4])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][4])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][4]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ])
                                                   ],flush=True)
                                               ], width=6),
                                               
                                               dbc.Col([
                                                   dbc.ListGroup([
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][5])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][5])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][5]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][6])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][6])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][6]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][7])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][7])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][7]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][8])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][8])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][8]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                       dbc.ListGroupItem([
                                                           dbc.Row([
                                                               dbc.Col([
                                                                   html.P(grade_card['columns'][9])
                                                               ], width=6, className="fw-bold"),
                                                               dbc.Col([
                                                                   html.P(grade_card['data'][0][9])
                                                               ], width=4),
                                                               dbc.Col([
                                                                   html.P(assign_grade(grade_card['data'][0][9]))
                                                               ], width=2)
                                                           ], className="text-muted fs-6", style={'font-family':'monospace', 'letter-spacing': '2px'})
                                                       ]),
                                                   ],flush=True)
                                               ], width=6),
                                               
                                               #Overall Grade(Based on Average)
                                               dbc.Col([
                                                   html.P(f"Overall Grade: {assign_grade(grade_card['data'][0][11])}" )
                                               ],width=12, className="fs-4 fw-bold", style={'font-family':'monospace', 'letter-spacing': '2px', 'color': '#132257'})
                                           ]),

                                           #Development Graph(Uses the Archived Sheet)
                                           dbc.Row([
                                               html.Hr(),
                                               dbc.Col([
                                                   html.H2("Development",className="m-b-20 text-bold text-center",
                                                           style={'font-family':'monospace', 'letter-spacing': '3px'}),
                                                   html.Hr(),
                                               ], className="p-3"),
                                               #Subject Selection
                                               dbc.Row([
                                                   dbc.Col([
                                                        html.Div([
                                                            dcc.Dropdown(
                                                                id='subject-dropdown-1',
                                                                options=[
                                                                    {'label': subject, 'value': subject} for subject in df.iloc[:,9:].columns
                                                                ],
                                                                value = 'Average', #Default Value
                                                                placeholder = 'Subject',
                                                                searchable = True
                                                            )
                                                        ], className="p-3"),
                                                       html.Div([
                                                           dcc.Dropdown(
                                                               id='subject-dropdown-2',
                                                               options=[
                                                                   {'label': subject, 'value': subject} for subject in df.iloc[:,9:].columns
                                                               ],
                                                               placeholder = 'Subject',
                                                               searchable = True
                                                           )
                                                       ],className="p-3"),
                                                       html.Div([
                                                           dcc.Dropdown(
                                                               id='subject-dropdown-3',
                                                               options=[
                                                                   {'label': subject, 'value': subject} for subject in df.iloc[:,9:].columns
                                                               ],
                                                               placeholder = 'Subject',
                                                               searchable = True
                                                           )
                                                       ],className="p-3")
                                                   ], width=3, style={'align-items': 'center'}),
                                                   dbc.Col([
                                                       dcc.Graph(
                                                           id='create-line-graph'
                                                       )
                                                   ], width=9)
                                               ], className="d-flex align-items-center")
                                           ])
                                       ])
                                    ])
                                ])
                            ], style={'display': 'flex', 'justify-content':'center', 'align-items':'center'}),
                        ],style={"max-width":"100%","overflow" :"hidden"}),
                    ],),
                ],justify="center"),
            ],),
        ],width=12)
    ])
    return card



app.layout = dbc.Container([
    html.Div('Student Analytics', style={
        'padding':'3px',
        'color':'#f0f2f3',
        'background-color':' #343d40',
        'font-size': '50px',
        'font-family': 'monospace',
        'letter-spacing': '2px',
        'display': 'flex',
        'justify-content': 'center',
        'margin-bottom': '20px',
        'border-radius': '10px'
    }),
    dbc.Row([
        #Student Selection Dropdown
        dbc.Col([
            dcc.Dropdown(
                id='dropdown-student',
                placeholder = 'Search Student',
                searchable = True
            ),
        ],width=6),

        #Class Selection Dropdown
        dbc.Col([
            dcc.Dropdown(
                id='dropdown-class',
                options=[
                    {'label': cls, 'value': cls} for cls in df['Class'].unique()
                ],
                placeholder = 'Search Class',
                searchable = True
            )
        ],width=6),
    ]),

    #Outpu Card Section
    html.Div([],  id = 'card-output', style={'margin-top':'40px'})
])


# Define callback to update output based on dropdown selection

#Select class, get only student selection of that class 
@app.callback(
    Output('dropdown-student', 'options'),
    Input('dropdown-class', 'value')
)
def update_student_dropdown(cls):
    if cls == None:
        return [{'label': (df[df['Student ID'] == std_id]['Name'].to_numpy()[0] +' (Class '+str(df[df['Student ID'] == std_id]['Class'].to_numpy()[0])+')'), 'value': std_id} for std_id in df['Student ID']]
    else:
        students = df[df['Class'] == cls]
        return [{'label': students[students['Student ID'] == std_id]['Name'].to_numpy()[0], 'value':std_id} for std_id in students['Student ID']]

#Select Student, get student data card
@app.callback(
    Output('card-output', 'children'),
    Input('dropdown-student', 'value'),
)
def update_output(std_id):
    card = createStudentCard(std_id)
    return card

# Select subjects inside the data card, get a line graph for that particular subject of that student
@app.callback(
    Output('create-line-graph', 'figure'),
    Input('subject-dropdown-1', 'value'),
    Input('subject-dropdown-2', 'value'),
    Input('subject-dropdown-3', 'value'),
    Input('dropdown-student', 'value'),
)
def update_development_graph_output(sub1, sub2, sub3, std_id):
    stud = df[df['Student ID'] == std_id]
    curData = stud[['Class', 'English', 'History', 'Geography', 'Communications', 'Physics', 'Chemistry', 'Mathematics', 'Biology', 'Computer', 'Physical Education','Total', 'Average']]
    #print(curData)
    arcData = df_archive[df_archive['Student ID'] == std_id][['Class', 'English', 'History', 'Geography', 'Communications', 'Physics', 'Chemistry', 'Mathematics', 'Biology', 'Computer', 'Physical Education', 'Total', 'Average']]
    #print(arcData)
    data = pd.concat([arcData, curData])
    if sub2 == None:
        if sub3 == None:
            fig = px.line(data, x = 'Class', y = sub1, markers=True)
        else:
            fig = px.line(data, x = 'Class', y = [sub1, sub3], markers=True)
    else:
        if sub3 == None:
            fig = px.line(data, x='Class', y =[sub1, sub2], markers=True)
        else:
            fig = px.line(data,x='Class', y=[sub1, sub2, sub3], markers=True)
            
    fig.update_layout(xaxis={'title': 'Class'},
                      yaxis = {'title': 'Marks'},
                      title = {'text': f'Development Graph'},
                      font = {'size': 10})
    fig.update_traces(patch={"line": {"width": 3}})
    return fig
        
    

if __name__ == "__main__":
    app.run_server(debug=True)




<h2 style="color: #f0f2f3; background-color: #343d40; font-size: 50px; font-family: monospace; letter-spacing: 2px; padding:20px;">Part 2: Class Analysis</h2>



In [30]:

def getOverallChart(stdInCls, subject = None):
    tmp_df = pd.DataFrame(columns = ['Marks', 'Count'])
    if subject == None:
        subject = 'Average'
    
    above90 = stdInCls[subject].between(90, 100, inclusive='both').sum()
    range80To90 = stdInCls[subject].between(80, 90, inclusive='left').sum()
    range70To80 = stdInCls[subject].between(70, 80, inclusive='left').sum()
    range60To70 = stdInCls[subject].between(60, 70, inclusive='left').sum()
    range50To60 = stdInCls[subject].between(50, 60, inclusive='left').sum()
    below50 = stdInCls[subject].between(0, 50, inclusive='left').sum()
    
    tmp_df['Marks'] = ['> 90', '80 - 90', '70 - 80', '60 - 70', '50 - 60', '< 50']
    tmp_df['Count'] = [above90, range80To90, range70To80, range60To70, range50To60, below50]
    
    fig = px.pie(tmp_df, 
                 values='Count',
                 names='Marks',
                 title=f"Total Students: {len(stdInCls)}",
                 hole=0.3,
                 color = 'Marks',
                 color_discrete_map={'> 90':'#5D3FD3',
                                     '80 - 90':'#0047AB',
                                     '70 - 80':'#0096FF',
                                     '60 - 70':'#6495ED',
                                     '50 - 60': '#89CFF0',
                                     '< 50': '#ADD8E6'},
                )
    fig.update_traces(textposition='inside', textinfo='label + percent', textfont={'size': 15, 'color': '#F0FFF0', 'family': 'monospace'})
    fig.update_layout(showlegend = False)
    fig.update_layout(margin=dict(t=100,b=0, l=0, r=0))
    
    return fig

def getHistoryGraph(cls):
    currYear = datetime.now().year
    new_dict = dict()
    current = df[df['Class'] == cls]
    new_dict[currYear] = round(current['Average'].mean(),2)
    
    count=cls
    while count != 12:
        currYear = currYear - 1
        prev = df_archive[df_archive['Class'] == cls]
        previousStats = prev[prev['Year'] == currYear]
        new_dict[currYear] = round(previousStats['Average'].mean(),2)
        count = count + 1
        
    d = {'Year': new_dict.keys(), 'Class Average': new_dict.values()}
    new_df = pd.DataFrame(d).dropna()
    
    fig3 = px.line(new_df, x = 'Year', y = 'Class Average', markers=True)
    fig3.update_layout(xaxis={'title': 'Session Year'},
                      yaxis = {'title': 'Class Average'},
                      title = {
                          'text': 'Class Progression',
                          'font':{'size':30, 'family':'monospace', 'color': '#757473'}},
                      font = {'size': 10})
    fig3.update_traces(patch={"line": {"width": 3}})
    
    return fig3


def createClassCard(cls):
    stdInCls = df[df['Class'] == cls]
    stdInCls = stdInCls.sort_values(by = ['Total'], ascending = False)
    
    #Rank 1:
    rank_1 = stdInCls.iloc[0,:][['Name', 'Profile Image', 'Total', 'Average']]
    
    #Rank 2:
    rank_2 = stdInCls.iloc[1,:][['Name', 'Profile Image', 'Total', 'Average']]
    
    #Rank 3:
    rank_3 = stdInCls.iloc[2,:][['Name', 'Profile Image', 'Total', 'Average']]
    
    
    card = dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader([
                        dbc.Row([
                            #Class Name Heading
                            dbc.Col([
                                html.H6(
                                    f"Class {cls}",
                                    className="display-5 m-b-20 text-center",
                                    style={'font-family': 'monospace','letter-spacing':'2px'}),
                            ], width=12)
                        ])
                    ], className="p-2 bg-dark text-white", style={'font-size': '15px'}),
                    
                    dbc.CardBody([
                        #Rankers
                        html.H2('Top Rankers', className='text-start'),
                        html.Hr(),
                        dbc.Row([
                            #Rank 1
                            dbc.Col([
                                dbc.Card([
                                    #Rank Badge
                                    dbc.Badge([
                                        html.H2("1", className="px-2 fs-3")
                                    ],
                                              color="dark",
                                              text_color="white",
                                              className="position-absolute translate-middle rounded-circle",
                                              style={'top':'24px', 'left':'8px'}
                                             ),
                                    #Student Name
                                    dbc.CardHeader([
                                       html.H2(rank_1['Name'], className="text-dark", style={'font-size':'13px'}),  
                                    ]),
                                    #Image, Avg. and Grade
                                    dbc.CardBody([
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.CardImg(src=rank_1['Profile Image'], className="m-b-25 img-radius")
                                            ], width=4),
                                            
                                            dbc.Col([
                                                dbc.Row([
                                                    dbc.Col([
                                                        html.H6(f"Avg.: {rank_1['Average']}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=11),
                                                    dbc.Col([
                                                        html.H6(f"Grade.: {assign_grade(rank_1['Average'])}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=10)
                                                ])
                                            ], width=8),

                                        ])
                                    ])
                                ], className="shadow-lg position-relative border-none")
                            ], width=4, className="p-4"),
                            
                            #Rank 2
                            dbc.Col([
                                dbc.Card([
                                    #Rank Badge
                                    dbc.Badge([
                                        html.H2("2", className="px-2 fs-3")
                                    ],
                                              color="dark",
                                              text_color="white",
                                              className="position-absolute translate-middle rounded-circle",
                                              style={'top':'24px', 'left':'8px'}
                                             ),
                                    #Student Name
                                    dbc.CardHeader([
                                        html.H2(rank_2['Name'], className="text-dark", style={'font-size':'13px'}),  
                                    ]),
                                    #Image, Avg. and Grade
                                    dbc.CardBody([
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.CardImg(src=rank_2['Profile Image'], className="m-b-25 img-radius")
                                            ], width=4),
                                            
                                            dbc.Col([
                                                dbc.Row([
                                                    dbc.Col([
                                                        html.H6(f"Avg.: {rank_2['Average']}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=11),
                                                    dbc.Col([
                                                        html.H6(f"Grade.: {assign_grade(rank_2['Average'])}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=10)
                                                ])
                                            ], width=8),
                                        ])
                                    ])
                                ], className="shadow-lg position-relative border-none")
                            ], width=4, className="p-4"),

                            #Rank 3
                            dbc.Col([
                                #Rank Badge
                                dbc.Card([
                                    dbc.Badge([
                                        html.H2("3", className="px-2 fs-3")
                                    ],
                                              color="dark",
                                              text_color="white",
                                              className="position-absolute translate-middle rounded-circle",
                                              style={'top':'24px', 'left':'8px'}
                                             ),
                                    #Student Name
                                    dbc.CardHeader([
                                       html.H2(rank_3['Name'], className="text-dark", style={'font-size':'13px'}),  
                                    ]),
                                    #Image, Avg. and Grade
                                    dbc.CardBody([
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.CardImg(src=rank_3['Profile Image'], className="m-b-25 img-radius")
                                            ], width=4),
                                            
                                            dbc.Col([
                                                dbc.Row([
                                                    dbc.Col([
                                                        html.H6(f"Avg.: {rank_3['Average']}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=11),
                                                    dbc.Col([
                                                        html.H6(f"Grade.: {assign_grade(rank_3['Average'])}", className="text-muted fw-bold", style={'font-size':'15px'})
                                                    ],width=10)
                                                ])
                                            ], width=8),
                                        ])
                                    ])
                                ], className="shadow-lg position-relative border-none")
                            ], width=4, className="p-4")
                        ]),

                        #Statistic Chart
                        dbc.Col([
                            html.Br(),
                            html.Br(),
                            html.H2("Statistics", className="text-start"),
                            html.Hr(),
                            html.Br(),
                            
                            #Pie Charts
                            dbc.Row([
                                dbc.Col([
                                    #Overall
                                    html.H4("Overall", className="text-start text-muted"),
                                    html.Hr(),
                                    html.Br(),
                                    dcc.Graph(figure=getOverallChart(stdInCls, 'Average'), className='py-4'),
                                    html.P(f'Chart showing the distribution of Average marks in the current class {cls}',
                                          style={'font-size': '12px'},  className="text-muted p-4"),
                                    html.H2(f'Class Average: {round(stdInCls['Average'].mean(),2)}', className="text-start text-muted", style={'color': '#132257'})
                                    
                                ], width=6),
                                
                                dbc.Col([
                                    #Subject Wise
                                    html.H4("Subject-Wise", className="text-start text-muted"),
                                    html.Hr(),
                                    dcc.Dropdown(
                                        id='dropdown-subject',
                                        options=[
                                            {'label': subject, 'value': subject} for subject in df.iloc[:,9:19].columns
                                        ],
                                        value='English',# Default value
                                        searchable = True
                                    ),
                                    
                                    #Chart Ouput Section
                                    dbc.Col([
                                        dcc.Graph(id = 'subject-chart-output', className="py-3"),
                                        html.P(id='subject-chart-describe', style={'font-size': '12px'},  className="text-muted p-4"),
                                        html.H2(id='subject-class-average', className="text-start text-muted", style={'color': '#132257'})
                                    ])
                                ], width=6)
                            ],style={})
                        ]),

                        #Historical Analysis of The Class
                        dbc.Col([
                            html.Br(),
                            html.Br(),
                            html.H2("History"),
                            html.Hr(),
                            dcc.Graph(figure = getHistoryGraph(cls))
                        ], className="p-3"),
                    ], style={'padding': '33px'})
                ], className="text-center", style={'font-family': 'monospace','letter-spacing':'2px'})
            ],{})
        ],{})
    ],{})
    
    return card

In [32]:
app2 = dash.Dash(
    __name__,external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True
)


app2.layout = dbc.Container([
    html.Div('Class Analytics', style={
        'padding':'3px',
        'color':'#f0f2f3',
        'background-color':' #343d40',
        'font-size': '50px',
        'font-family': 'monospace',
        'letter-spacing': '2px',
        'display': 'flex',
        'justify-content': 'center',
        'margin-bottom': '20px',
        'border-radius': '10px'
    }),
    dbc.Row([
        dbc.Col([
            #Class Selection Dropdown
            dcc.Dropdown(
                id='dropdown-class',
                options=[
                    {'label': f'Class {cls}', 'value': cls} for cls in df['Class'].unique()
                ],
                placeholder = 'Search Class',
                searchable = True
            )
            ],width=10),
        ]),
    
    #Output Data Card
    html.Div([],  id = 'class-card-output', style={'margin-top':'40px'})
])

# Define callback to update output based on dropdown selection

# Select class, get class data card
@app2.callback(
    Output('class-card-output', 'children'),
    Input('dropdown-class', 'value')
)
def update_output(cls):
    if cls == None:
        return None
    card = createClassCard(cls)
    return card

#Select subject, get pie chart distribution, a small description, and class Avg. 
#of marks in that subject in that class

@app2.callback(
    Output('subject-chart-output', 'figure'),
    Output('subject-chart-describe', 'children'),
    Output('subject-class-average', 'children'),
    Input('dropdown-subject', 'value'),
    Input('dropdown-class', 'value')
)
def update_subject_specific_chart(subject, cls):
    stdInCls = df[df['Class'] == cls]
    if subject == None:
        return getOverallChart(stdInCls, subject), '  ', '  '
    cls_avg = f'Class Average: {round(stdInCls[subject].mean(),2)}'
    chart_description = f'Chart showing the distribution of {subject}  marks in the current class {cls}'
    return getOverallChart(stdInCls, subject), chart_description, cls_avg


if __name__ == '__main__':
    app2.run_server(debug = True)



<h2 style="color: #f0f2f3; background-color: #343d40; font-size: 50px; font-family: monospace; letter-spacing: 2px; padding:20px;">Part 3: Student Comparison</h2>



In [34]:
def getDevCompareGraph(student1, student2):
    pio.templates.default = 'seaborn'
    curData1 = student1[['Student ID', 'Class', 'Average']]
    arcData1 = df_archive[df_archive['Student ID'] == student1['Student ID'].to_numpy()[0]][['Student ID', 'Class', 'Average']]
    
    curData2 = student2[['Student ID', 'Class', 'Average']]
    arcData2 = df_archive[df_archive['Student ID'] == student2['Student ID'].to_numpy()[0]][['Student ID', 'Class', 'Average']]
    
    data = pd.concat([arcData1, curData1, arcData2, curData2])
    
    data['Student ID'] = data['Student ID'].replace([student1['Student ID'], student2['Student ID']], [student1['Name'], student2['Name']])
    data = data.rename(columns={'Student ID': 'Name'})
    #print(data)
    
    fig = px.line(data, x='Class', y='Average', color='Name', markers=True)        
    fig.update_layout(xaxis={'title': 'Class'},
                      yaxis = {'title': 'Average Marks'},
                      font = {'size': 10, 'family':'monospace'},
                      legend={'font_size': 10},
                      margin = {'t': 0, 'b': 0, 'l': 0, 'r': 0}
                      )
    fig.update_traces(patch={"line": {"width": 3}})
    
    return fig

def createSubjectComparisonCard(subject, student1, student2):
    curData1 = student1[['Student ID', 'Class', subject]]
    arcData1 = df_archive[df_archive['Student ID'] == student1['Student ID'].to_numpy()[0]][['Student ID', 'Class', subject]]
    curData2 = student2[['Student ID', 'Class', subject]]
    arcData2 = df_archive[df_archive['Student ID'] == student2['Student ID'].to_numpy()[0]][['Student ID', 'Class', subject]]
    
    dff = pd.concat([arcData1, curData1, arcData2, curData2])
    
    dff['Student ID'] = dff['Student ID'].replace([student1['Student ID'], student2['Student ID']], [student1['Name'], student2['Name']])
    dff = dff.rename(columns={'Student ID': 'Name'})
    #print(dff)
    
    fig = px.line(dff, x='Class', y=subject, color='Name', markers=True)        
    fig.update_layout(xaxis={'title': 'Class'},
                      yaxis = {'title': 'Average Marks'},
                      font = {'size': 10, 'family':'monospace'},
                      legend = {'font_size': 10},
                      margin = {'t': 0, 'b': 0, 'l': 0, 'r': 0})
    fig.update_traces(patch={"line": {"width": 3}})
    
    return fig

def createDevGraph(student):
    curData = student[['Class', 'Average']]
    arcData = df_archive[df_archive['Student ID'] == student['Student ID'].to_numpy()[0]][['Class', 'Average']]
    
    data = pd.concat([arcData, curData])
    
    fig = px.line(data, x='Class', y='Average', markers=True)        
    fig.update_layout(xaxis={'title': 'Class'},
                      yaxis = {'title': 'Average Marks'},
                      font = {'size': 10, 'family':'monospace'},
                     margin = {'t': 0, 'b': 0, 'l': 0, 'r': 0})
    fig.update_traces(patch={"line": {"width": 3}})
    
    return fig

In [36]:
def createComparisonCard(std1, std2):
    student1 = df[df['Student ID'] == std1]
    student2 = df[df['Student ID'] == std2]

    strength11, strength12, strength13 = strengths(student1)
    strength21, strength22, strength23 = strengths(student2)
    weakness11, weakness12, weakness13 = weaknesses(student1)
    weakness21, weakness22, weakness23 = weaknesses(student2)

    #Comparison Data Card
    comparison_card = dbc.Card([
        dbc.CardBody([
            dbc.Row([
                #Student 1
                dbc.Col([
                   dbc.CardBody([
                       dbc.Row([
                           #Image
                           dbc.Col([
                               dbc.CardImg(src = student1['Profile Image'].to_numpy()[0], className="img-radius")
                           ], style={}, width=3),
                           #Name
                           dbc.Col([
                               html.H4(student1['Name'], className="fs-4"),
                           ], style={}, width=9)
                       ], style={'background-color': '#132257', 'color':'#f0f2f3'}, className="p-2 rounded"),
                       html.Br(),
                       html.H5('Details', className="text-muted"),
                       html.Hr(),

                       #Details - Class Rank and Avg.
                       dbc.Row([
                           dbc.Col([
                              dbc.CardHeader([
                                   html.H5('Class', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(student1['Class'].to_numpy()[0], className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'})
                           ], width=4),
                           dbc.Col([
                              dbc.CardHeader([
                                   html.H5('Rank', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(calculateRank(student1), className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'})
                           ], width=4),
                           dbc.Col([
                              dbc.CardHeader([
                                   html.H5('Avg.', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(f"{int(round(student1['Average'].to_numpy()[0]))}%", className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'})
                           ], width=4),
                       ]),
                       html.Br(),
                       html.H5('Strengths', className="text-muted"),
                       html.Hr(),
                       
                       #Strengths
                       dbc.Col([
                           html.P(
                               strength11[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength11[1])}",
                               value=round(strength11[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               strength12[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength12[1])}",
                               value=round(strength12[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               strength13[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength13[1])}",
                               value=round(strength13[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"}),
                       ], className="p-3"),
                       html.Br(),
                       html.H5('Weaknesses', className="text-muted"),
                       html.Hr(),
                       #Weaknesses
                       dbc.Col([
                           html.P(
                               weakness11[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness11[1])}",
                               value=round(weakness11[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               weakness12[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness12[1])}",
                               value=round(weakness12[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               weakness13[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness13[1])}",
                               value=round(weakness13[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"})
                       ], className="p-3"),
                   ], className="shadow-lg p-5 m-1")
                ], style={}, width=6),

                #Student 2
                dbc.Col([
                   dbc.CardBody([
                       dbc.Row([
                           #Image
                           dbc.Col([
                               dbc.CardImg(src = student2['Profile Image'].to_numpy()[0], className="img-radius")
                           ], style={}, width=3),
                           #Name
                           dbc.Col([
                               html.H4(student2['Name'], className="fs-4"),
                           ], style={}, width=9)
                       ], style={'background-color': '#132257', 'color':'#f0f2f3'}, className="p-2 rounded"),
                       html.Br(),
                       html.H5('Details', className="text-muted"),
                       html.Hr(),
                       
                       #Details - Class, Rank and Avg.
                       dbc.Row([
                           dbc.Col([
                              dbc.CardHeader([
                                   html.H5('Class', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(student2['Class'].to_numpy()[0], className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'})
                           ], width=4),
                           dbc.Col([
                               dbc.CardHeader([
                                   html.H5('Rank', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(calculateRank(student2), className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'}),
                           ], width=4),
                           dbc.Col([
                               dbc.CardHeader([
                                   html.H5('Avg.', className="card-subtitle text-start", style={'font-size': '10px', 'margin-bottom':'7px'}),
                                   html.Hr(className="p-0 m-0"),
                                   html.H4(f"{int(round(student2['Average'].to_numpy()[0]))}%", className='card-text text-start mt-1 ms-1 fs-4 fw-bold')
                                   ], className="rounded p-2", style={'background-color': '#6CABDD', 'color': '#ffffff'})
                           ], width=4),
                       ]),
                       html.Br(),
                       
                       html.H5('Strengths', className="text-muted"),
                       html.Hr(),
                       #Strengths
                       dbc.Col([
                           html.P(
                               strength21[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength21[1])}",
                               value=round(strength21[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               strength12[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength22[1])}",
                               value=round(strength22[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               strength13[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(strength23[1])}",
                               value=round(strength23[1]),
                               min=0,
                               max=100,
                               color="#132257",
                               style={"height": "15px"})
                       ], className="p-3"),
                       html.Br(),
                       html.H5('Weaknesses', className="text-muted"),
                       html.Hr(),
                       #Weaknesses
                       dbc.Col([
                           html.P(
                               weakness21[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness21[1])}",
                               value=round(weakness21[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               weakness22[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness22[1])}",
                               value=round(weakness22[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"}),
                           html.Br(),
                           html.P(
                               weakness23[0],
                               className="f-w-600 text-bold",
                               style = {'font-family':'monospace', 'letter-spacing': '2px', 'color': 'grey', 'font-size':'15px'}),
                           dbc.Progress(
                               label=f"{round(weakness23[1])}",
                               value=round(weakness23[1]),
                               min=0,
                               max=100,
                               color="#6CABDD",
                               style={"height": "15px"}),
                       ], className="p-3"),
                   ], className="shadow-lg p-5 m-1")
                ], style={}, width=6)
            ], style={}, className=""),
            html.Br(),
            html.Br(),
            
            dbc.Row([
                #Heading
                dbc.Col([
                    html.H2('Comparison'),
                    html.Hr(),
                    html.Br(),
                ], width=12),
                
                dbc.Row([
                    #Development (Avg. vs Avg. over the years) 
                    dbc.Col([
                        html.H4('Development'),
                        html.Hr(),
                        #Line Graph
                        dcc.Graph(figure = getDevCompareGraph(student1, student2)),
                        html.Br(),
                    ],style={}, width=12),
                    html.Br(),
                    
                    #Development (marks in selected subject vs marks in selected subject over the years)
                    dbc.Col([
                        html.H4('Subject-wise'),
                        html.Hr(),
                        #Subject Selection Dropdown
                        dcc.Dropdown(
                            id='subject-dropdown',
                            options=[
                                 {'label': subject, 'value': subject} for subject in df.iloc[:,9:19].columns
                            ],
                            value = 'English',# Default value
                            searchable = True
                        ),
                        html.Br(),
                        #Line Graph
                        dcc.Graph(id = 'subject-compare-graph'),
                    ],style={}, width=12),
                ], style={}, className="text-start text-muted")
            ], style={}, className="text-center")
        ], style={}, className="p-3")
    ], style={}, className="shadow-lg p-3")
   
    return comparison_card

In [38]:
app5 = dash.Dash(
    __name__,external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True
)

app5.layout = [
    dbc.Container([
    dbc.Row([
        html.Div('Compare Students', style={
                'padding':'3px',
                'color':'#f0f2f3',
                'background-color':' #343d40',
                'font-size': '50px',
                'font-family': 'monospace',
                'letter-spacing': '2px',
                'display': 'flex',
                'justify-content': 'center',
                'margin-bottom': '20px',
                'border-radius': '10px'
            }),
        dbc.Col([
            #Student Selection Dropdown
            dcc.Dropdown(
                id='student-dropdown-1',
                options=[
                    {'label': df[df['Student ID'] == std]['Name'].to_numpy()[0] +' (Class '+str(df[df['Student ID'] == std]['Class'].to_numpy()[0])+')' , 'value': std} for std in df['Student ID']        
                ],
                placeholder = 'First Student',
                searchable = True
            ),
        ],width=6),
        dbc.Col([
            #Student Selection Dropdown
            dcc.Dropdown(
                id='student-dropdown-2',
                options=[
                    {'label': df[df['Student ID'] == std]['Name'].to_numpy()[0] +' (Class '+str(df[df['Student ID'] == std]['Class'].to_numpy()[0])+')' , 'value': std} for std in df['Student ID']        
                ],
                placeholder = 'Second Student',
                searchable = True
            ),
        ],width=6),
    ], style={'padding-top': '15px'}),
        
        #Comparison Data Card Output Section
        html.Div([],  id = 'card-section', style={'margin-top':'40px', 'font-family':'monospace', 'letter-spacing': '3px'})
    ])
]

# Define callback to update output based on dropdown selection

#Select 2 students, get comparison data card.
#Note - Selecting only 1 student doesn't generate the card, 2 students have to be selected
@app5.callback(
    Output('card-section', 'children'),
    Input('student-dropdown-1', 'value'),
    Input('student-dropdown-2', 'value')
)
def update_output(std1, std2):
    if std1 == None or std2 == None:
        return ' '
    comparisonCard = createComparisonCard(std1, std2)
    return comparisonCard

#Select a subject, get line graph for marks in that subject for the 2 students over the years
@app5.callback(
    Output('subject-compare-graph', 'figure'),
    Input('subject-dropdown', 'value'),
    Input('student-dropdown-1', 'value'),
    Input('student-dropdown-2', 'value')
)
def update_subject_compare_graph(subject, std1, std2):
    if std1 == None or std2 == None:
        return ' '
    student1 = df[df['Student ID'] == std1]
    student2 = df[df['Student ID'] == std2]
    return createSubjectComparisonCard(subject, student1, student2)

if __name__ == '__main__':
    app5.run_server(debug=True)